<a href="https://colab.research.google.com/github/lachlandeer/2016-04-07-FederalReserveBoard/blob/FirstLoad/action_cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import auth
auth.authenticate_user()

## Get Data From Google Cloud Storage

In [3]:
project_id = 'egypt-riot-network'

In [4]:
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://egypt-riot-network/


In [5]:
!mkdir data


In [7]:
!gsutil cp gs://egypt-riot-network/src_data/action_training/* data/.

Copying gs://egypt-riot-network/src_data/action_training/action_coder_01.xls...
Copying gs://egypt-riot-network/src_data/action_training/translations_coder2_ARJUN.xls...
\ [2 files][  7.8 MiB/  7.8 MiB]                                                
Operation completed over 2 objects/7.8 MiB.                                      


In [9]:
!ls data

action_coder_01.xls  translations_coder2_ARJUN.xls


## Read in Data

In [10]:
df_1 = pd.read_excel('data/action_coder_01.xls')
df_2 = pd.read_excel('data/translations_coder2_ARJUN.xls')

In [11]:
cols_keep = ['tweet_id', 'tweet', 'classification']
df_1 = df_1[cols_keep]
df_2 = df_2[cols_keep]

In [12]:
# add col to identify coder
df_1['coder'] = 'coder_01'
df_2['coder'] = 'coder_02'

In [13]:
df_1["classification"] = df_1["classification"].astype('category')
df_2["classification"] = df_2["classification"].astype('category')

## Collect aggregate dataframe to classify

In [15]:
# we use all of coder 1's classifications, and those by coder 2 which coder 1 didn't classify
agg_df = pd.merge(
                    df_1, df_2,
                    how='outer', 
                    on='tweet_id', 
                    indicator=True, 
                    suffixes=('_foo','')
                    )

In [16]:
agg_df.head()

,tweet_id,tweet_foo,classification_foo,tweet,classification,_merge
0,359447956342046720,بيقولك البلتاجي خاف يعين وزير دفاع لاشارة رابع...,PRO_INTERVENTION,NaN,NaN,left_only
1,365115804276105216,الببلاوي: قرار فض اعتصامي رابعة والنهضة نهائي ...,PRO_INTERVENTION,NaN,NaN,left_only
2,354583432807985152,رئاسة الجمهورية تعرض على دكتور حازم الببلاوى م...,IRRELEVANT,NaN,NaN,left_only
3,356152710241861632,@Dr_FAlShaiji دكتوره فاطمه رأيك في خطوه الجيش ...,NEUTRAL,NaN,NaN,left_only
4,357964339187167232,قلت له أكيد سباك قالي مرسي دكتور مش هيعرف يصلح...,PRO_INTERVENTION,NaN,NaN,left_only


In [17]:
coder2_only = agg_df.query('_merge == "right_only"')
coder2_only = coder2_only[cols_keep]

combine coder 1 and the coder 2 only data to one dataset

In [19]:
df = pd.concat([df_1, coder2_only])

In [23]:
df.head()

,tweet_id,tweet,classification
0,359447956342046720,بيقولك البلتاجي خاف يعين وزير دفاع لاشارة رابع...,PRO_INTERVENTION
1,365115804276105216,الببلاوي: قرار فض اعتصامي رابعة والنهضة نهائي ...,PRO_INTERVENTION
2,354583432807985152,رئاسة الجمهورية تعرض على دكتور حازم الببلاوى م...,IRRELEVANT
3,356152710241861632,@Dr_FAlShaiji دكتوره فاطمه رأيك في خطوه الجيش ...,NEUTRAL
4,357964339187167232,قلت له أكيد سباك قالي مرسي دكتور مش هيعرف يصلح...,PRO_INTERVENTION


In [25]:
df.shape

(5406, 3)

## Restrict data to binary classification

We only want tweets that are pro or anti-intervention

In [24]:
df = df.query('classification == "PRO_INTERVENTION"  \
         or classification == "ANTI_INTERVENTION"')

In [28]:
df.head()

,tweet_id,tweet,classification
0,359447956342046720,بيقولك البلتاجي خاف يعين وزير دفاع لاشارة رابع...,PRO_INTERVENTION
1,365115804276105216,الببلاوي: قرار فض اعتصامي رابعة والنهضة نهائي ...,PRO_INTERVENTION
4,357964339187167232,قلت له أكيد سباك قالي مرسي دكتور مش هيعرف يصلح...,PRO_INTERVENTION
6,382792082588700672,#شعر انا راسي مرسى الحقايق من الأوهام/ ماأرد ا...,ANTI_INTERVENTION
7,351666260666355712,استقالة وزير السياحة، وزير الاتصالات، وزير الب...,PRO_INTERVENTION


In [29]:
df.shape

(5406, 3)

In [30]:
df.classification.unique()

['PRO_INTERVENTION', 'ANTI_INTERVENTION']
Categories (2, object): ['PRO_INTERVENTION', 'ANTI_INTERVENTION']

In [31]:
df.groupby(['classification']).size()


classification
ANTI_INTERVENTION    3331
BAD_RECORD              0
IRRELEVANT              0
NEUTRAL                 0
PRO_INTERVENTION     2075
dtype: int64

## Save data to csv

In [20]:
!mkdir out_data

In [21]:
df.to_csv('out_data/action_handcoded_all.csv')

## Also create a balanced sample and save

In [22]:
!pip install imblearn


In [32]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy = 'not minority', random_state = 123456789)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [34]:
df_balanced, balanced_labels = rus.fit_resample(df, df['classification'])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [35]:
## convert back to dataframe

df_balanced_pd = pd.DataFrame(data = df_balanced, 
                              columns= cols_keep
                              )

In [36]:
df_balanced_pd.head()

,tweet_id,tweet,classification
0,353152423960907776,لو كان هدفك الأسمى مش انقلاب ليه جيت على إرادت...,ANTI_INTERVENTION
1,359969196932743168,وهو وافق عليها ورغم كده انقلبت يا كذاب RT @aa_...,ANTI_INTERVENTION
2,381856132873977920,على جمعه يتهم الاخوان بتعليم ابنائهم التكفير ا...,ANTI_INTERVENTION
3,382257999047163904,ابتسم:جون ماكين لعمرو أديب: ماحصل انقلاب؛انت ل...,ANTI_INTERVENTION
4,367720317554544640,@TheArabHash #هولوكست_السيسي #فرعون_وجنوده,ANTI_INTERVENTION


In [37]:
df.to_csv('out_data/action_handcoded_balanced.csv')

In [39]:
!gsutil cp out_data/*.csv gs://egypt-riot-network/out/data/actions/

Copying file://out_data/action_handcoded_all.csv [Content-Type=text/csv]...
Copying file://out_data/action_handcoded_balanced.csv [Content-Type=text/csv]...
\
Operation completed over 2 objects/3.5 MiB.                                      
